## 0. Importar librerias para dibujar landmarks

In [75]:
import mediapipe as mp
import cv2 

In [76]:
# Drawing helpers and Mediapipe Solutions
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

## 1. Dibujar landmarks con mediapipe 

In [77]:
# Inicializa la captura de video desde la webcam
cap = cv2.VideoCapture(0)

# Inicia el modelo Holistic de mediapipe para la detección de landmarks
# Los argumentos  min_detection_confidence y min_tracking_confidence  establecen los umbrales de confianza mínimos requeridos 
#para la detección y el seguimiento de landmarks en una imagen.
#min_detection_confidence: Este argumento establece el umbral de confianza mínimo necesario para que el modelo Holistic detecte un landmark en la imagen. Un valor más alto significa que el modelo solo detectará landmarks con una confianza muy alta, mientras que un valor más bajo permitirá detectar landmarks con una confianza más baja.
#min_tracking_confidence: Este argumento establece el umbral de confianza mínimo para el seguimiento de landmarks en imágenes sucesivas. Una vez que un landmark ha sido detectado en una imagen con una confianza superior a este umbral, el modelo intentará seguir ese landmark en imágenes posteriores. Un valor más alto aquí significa que el seguimiento se realizará solo para landmarks muy confiables, mientras que un valor más bajo permitirá un seguimiento incluso para landmarks menos confiables.
# Factore: buena iluminación, fondo simple, si se requiere precision : usar valores mas alto
# Un valor muy alto de confianza puede conducir a una menor cantidad de landmarks detectados y seguidos, lo que puede ser aceptable en algunos casos pero limitante en otros donde se necesite información más detallada.
#min_detection_confidence: deteccion
#min_tracking_confidence: seguimiento
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: #probar: min_detection_confidence=0.7-0.9, min_tracking_confidence=0.5-0.8
    while cap.isOpened():  # Bucle principal para procesar cada fotograma de video
        ret, frame = cap.read()  # Lee un fotograma de la webcam
        
        # Recolor Feed
        # Convierte el fotograma de BGR a RGB para el procesamiento de mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False   # Marca la imagen como no modificable   
        
        # Make Detections
        # Procesa la imagen utilizando el modelo Holistic para detectar landmarks
        results = holistic.process(image)
        # print(results.face_landmarks) # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Muestra la imagen con los landmarks dibujados en una ventana
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


## 0. Importar librerias para dibujar landmarks

In [ ]:
import mediapipe as mp
import cv2 

In [ ]:
# Drawing helpers and Mediapipe Solutions
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

## 1. Dibujar landmarks con mediapipe 

In [ ]:
# Inicializa la captura de video desde la webcam
cap = cv2.VideoCapture(0)

# Inicia el modelo Holistic de mediapipe para la detección de landmarks
# Los argumentos  min_detection_confidence y min_tracking_confidence  establecen los umbrales de confianza mínimos requeridos 
#para la detección y el seguimiento de landmarks en una imagen.
#min_detection_confidence: Este argumento establece el umbral de confianza mínimo necesario para que el modelo Holistic detecte un landmark en la imagen. Un valor más alto significa que el modelo solo detectará landmarks con una confianza muy alta, mientras que un valor más bajo permitirá detectar landmarks con una confianza más baja.
#min_tracking_confidence: Este argumento establece el umbral de confianza mínimo para el seguimiento de landmarks en imágenes sucesivas. Una vez que un landmark ha sido detectado en una imagen con una confianza superior a este umbral, el modelo intentará seguir ese landmark en imágenes posteriores. Un valor más alto aquí significa que el seguimiento se realizará solo para landmarks muy confiables, mientras que un valor más bajo permitirá un seguimiento incluso para landmarks menos confiables.
# Factore: buena iluminación, fondo simple, si se requiere precision : usar valores mas alto
# Un valor muy alto de confianza puede conducir a una menor cantidad de landmarks detectados y seguidos, lo que puede ser aceptable en algunos casos pero limitante en otros donde se necesite información más detallada.
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: #probar: min_detection_confidence=0.7-0.9, 

    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
results

mediapipe.python.solution_base.SolutionOutputs

In [ ]:
results.pose_landmarks

landmark {
  x: 0.5809244
  y: 0.24712658
  z: -0.739278
  visibility: 0.99993914
}
landmark {
  x: 0.6049273
  y: 0.15868685
  z: -0.6839315
  visibility: 0.99988085
}
landmark {
  x: 0.6220694
  y: 0.15958098
  z: -0.68387604
  visibility: 0.99987334
}
landmark {
  x: 0.63892114
  y: 0.16115968
  z: -0.6841286
  visibility: 0.99987036
}
landmark {
  x: 0.54849285
  y: 0.15705356
  z: -0.6899874
  visibility: 0.9998952
}
landmark {
  x: 0.52555025
  y: 0.15668614
  z: -0.68933886
  visibility: 0.9998963
}
landmark {
  x: 0.50590986
  y: 0.15615633
  z: -0.689562
  visibility: 0.9999029
}
landmark {
  x: 0.6626643
  y: 0.2011099
  z: -0.35323778
  visibility: 0.9998439
}
landmark {
  x: 0.47565588
  y: 0.19750349
  z: -0.3682824
  visibility: 0.99991006
}
landmark {
  x: 0.6106254
  y: 0.33324215
  z: -0.61935574
  visibility: 0.9999331
}
landmark {
  x: 0.54190326
  y: 0.33145252
  z: -0.6230754
  visibility: 0.9999421
}
landmark {
  x: 0.77338624
  y: 0.67101187
  z: -0.19209817
  vi

In [ ]:
results.pose_landmarks.landmark[0]

x: 0.5809244
y: 0.24712658
z: -0.739278
visibility: 0.99993914

In [ ]:
results.pose_landmarks.landmark[0].visibility

0.9999391436576843

In [ ]:
results.face_landmarks.landmark[0]

x: 0.56733704
y: 0.33550656
z: -0.0311259

In [ ]:
results.face_landmarks.landmark[0].visibility # visibility muestra si el landmark si se muestra en la pantalla o no
# para los landmarks de la cara siempre sera cero

0.0

<img src="https://i.imgur.com/AzKNp7A.png">

## 2. Capturar Landmarks & Exportar CSV

In [ ]:
import csv
import os
import numpy as np

In [ ]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_coords #501

501

In [ ]:
#Create header table | class | x | y | z | visibility |
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
len(landmarks) #2005

2005

In [ ]:
#Export header CSV
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## 3. Tomar coordenadas de señas

In [ ]:
#Save coord poses
class_name="Buuu que mal"

#Detectar puntos de referencia
cap = cv2.VideoCapture(0)  #0 o 2

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        # 1. Draw face landmarks
       # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
       #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
       #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
       #                          )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates, da todos los landmarks y los almacena en una matriz
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Concate rows
            row = pose_row+face_row

            # Append class name
            row.insert(0, class_name)

            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass


        cv2.imshow('Rosa Webcam', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
results.pose_landmarks.landmark[0]

x: 0.5739429
y: 0.31865352
z: -0.85075855
visibility: 0.9998646

In [ ]:
results.face_landmarks.landmark[0].visibility #la visibilidad de face_landmark siempre sera cero

0.0

Numero de coordenadas que tendremos:

In [ ]:
pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
len(pose_row) #132


132

In [ ]:
face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
len(face_row)

1872

In [ ]:
row=pose_row+face_row #total de coordenadas deberia de ser 2004
len(row)


2004

In [ ]:
1+501*4

2005

In [ ]:
row.insert(0, class_name) #agregar el nombre de la clase (pose)
len(row) #1+501*4 class_name, landmarks, header (x,y,z,visibility)

2005

## 4. Entrenar modelo usando Scikit Learn

### Leer y recolectar datos

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Buuieeen,0.530443,0.256885,-0.653312,0.999986,0.559472,0.183092,-0.591619,0.999977,0.577272,...,0.003441,0.0,0.577638,0.191048,0.022638,0.0,0.582679,0.182326,0.023350,0.0
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0


In [ ]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
542,Buuu que mal,0.561161,0.312676,-0.954139,0.999895,0.585262,0.240363,-0.879301,0.999777,0.600595,...,0.002703,0.0,0.612643,0.256506,0.021253,0.0,0.617767,0.247276,0.021986,0.0
543,Buuu que mal,0.561202,0.313547,-1.004511,0.999883,0.585302,0.240690,-0.925527,0.999742,0.600615,...,0.002702,0.0,0.616873,0.256113,0.021999,0.0,0.621815,0.247989,0.022703,0.0
544,Buuu que mal,0.564997,0.315829,-0.923724,0.999874,0.587699,0.241852,-0.852775,0.999715,0.603202,...,0.003340,0.0,0.620147,0.253019,0.022850,0.0,0.625026,0.244550,0.023674,0.0
545,Buuu que mal,0.570446,0.317763,-0.891257,0.999865,0.592185,0.243242,-0.811182,0.999696,0.608082,...,0.002278,0.0,0.621281,0.251643,0.021229,0.0,0.626135,0.244106,0.021871,0.0
546,Buuu que mal,0.573943,0.318654,-0.850759,0.999865,0.594787,0.243582,-0.771189,0.999693,0.610638,...,0.002614,0.0,0.622002,0.249436,0.022121,0.0,0.626846,0.241835,0.022813,0.0


In [ ]:
df[df['class']=='Bieeen']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0
5,Bieeen,0.524074,0.269810,-0.796441,0.999970,0.551455,0.191224,-0.731950,0.999950,0.567993,...,0.003378,0.0,0.574250,0.191542,0.022691,0.0,0.579120,0.183588,0.023304,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,Bieeen,0.502346,0.364458,-0.711878,0.999970,0.523844,0.285156,-0.645986,0.999950,0.538261,...,0.000084,0.0,0.568144,0.287267,0.020360,0.0,0.573151,0.280079,0.021128,0.0
333,Bieeen,0.504117,0.367366,-0.690439,0.999969,0.525992,0.288800,-0.626301,0.999947,0.540995,...,0.000711,0.0,0.572651,0.290650,0.022053,0.0,0.578006,0.282941,0.022980,0.0
334,Bieeen,0.504405,0.366832,-0.775375,0.999964,0.526557,0.287651,-0.712337,0.999936,0.541663,...,0.001722,0.0,0.570707,0.279155,0.022135,0.0,0.576245,0.269708,0.023198,0.0
335,Bieeen,0.504425,0.360905,-0.769675,0.999963,0.527333,0.283022,-0.703782,0.999935,0.542733,...,0.000002,0.0,0.569716,0.265240,0.020307,0.0,0.575013,0.256650,0.021239,0.0


x, y, z son las caracteristicas que le pasaremos al modelo para entrenarlo.
La idea es que se pasen esas coordenadas y el modelo pueda predecir que clase es (pose)

In [ ]:
X = df.drop('class', axis=1) # coordenadas
y = df['class'] # clase

In [ ]:
y #X = coords , y = class

0          Buuieeen
1            Bieeen
2            Bieeen
3            Bieeen
4            Bieeen
           ...     
542    Buuu que mal
543    Buuu que mal
544    Buuu que mal
545    Buuu que mal
546    Buuu que mal
Name: class, Length: 547, dtype: object

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
444,0.607627,0.331788,-0.786915,0.999879,0.629459,0.266417,-0.717988,0.999843,0.644549,0.268743,...,0.001988,0.0,0.665968,0.260837,0.021296,0.0,0.670673,0.253972,0.021934,0.0
480,0.561522,0.337758,-0.722480,0.999941,0.583754,0.270509,-0.622497,0.999925,0.598792,0.271359,...,0.004055,0.0,0.608456,0.254144,0.021859,0.0,0.613111,0.245462,0.022605,0.0
134,0.505146,0.288510,-0.693865,0.999984,0.529100,0.220908,-0.624870,0.999972,0.543936,0.223276,...,0.002321,0.0,0.554203,0.209690,0.022227,0.0,0.558938,0.202911,0.022755,0.0
93,0.512465,0.361475,-0.661181,0.999947,0.533841,0.292627,-0.601996,0.999917,0.547602,0.292967,...,0.001359,0.0,0.565009,0.273539,0.020338,0.0,0.569777,0.266539,0.020884,0.0
363,0.538592,0.274794,-0.750494,0.999549,0.565521,0.204791,-0.675919,0.999103,0.580280,0.206831,...,0.004653,0.0,0.594222,0.208484,0.024701,0.0,0.599227,0.200359,0.025415,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.488968,0.363467,-0.626467,0.999947,0.515220,0.283686,-0.562118,0.999918,0.531321,0.281666,...,-0.002455,0.0,0.550067,0.273457,0.016614,0.0,0.555085,0.266556,0.017167,0.0
372,0.553317,0.296857,-0.723984,0.999655,0.575554,0.220616,-0.667189,0.999247,0.591970,0.219673,...,0.003934,0.0,0.607926,0.218048,0.023417,0.0,0.612781,0.210321,0.024091,0.0
204,0.480797,0.354897,-0.773540,0.999975,0.500139,0.278386,-0.703254,0.999961,0.516196,0.275988,...,0.001624,0.0,0.530649,0.266843,0.021341,0.0,0.535038,0.260296,0.022014,0.0
53,0.520480,0.353810,-0.560121,0.999949,0.541462,0.283457,-0.492140,0.999928,0.554404,0.284279,...,0.002891,0.0,0.569414,0.277148,0.022025,0.0,0.574226,0.270020,0.022672,0.0


In [ ]:
df #marco de datos completo, deben de ser 548 filas

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Buuieeen,0.530443,0.256885,-0.653312,0.999986,0.559472,0.183092,-0.591619,0.999977,0.577272,...,0.003441,0.0,0.577638,0.191048,0.022638,0.0,0.582679,0.182326,0.023350,0.0
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,Buuu que mal,0.561161,0.312676,-0.954139,0.999895,0.585262,0.240363,-0.879301,0.999777,0.600595,...,0.002703,0.0,0.612643,0.256506,0.021253,0.0,0.617767,0.247276,0.021986,0.0
543,Buuu que mal,0.561202,0.313547,-1.004511,0.999883,0.585302,0.240690,-0.925527,0.999742,0.600615,...,0.002702,0.0,0.616873,0.256113,0.021999,0.0,0.621815,0.247989,0.022703,0.0
544,Buuu que mal,0.564997,0.315829,-0.923724,0.999874,0.587699,0.241852,-0.852775,0.999715,0.603202,...,0.003340,0.0,0.620147,0.253019,0.022850,0.0,0.625026,0.244550,0.023674,0.0
545,Buuu que mal,0.570446,0.317763,-0.891257,0.999865,0.592185,0.243242,-0.811182,0.999696,0.608082,...,0.002278,0.0,0.621281,0.251643,0.021229,0.0,0.626135,0.244106,0.021871,0.0


### Modelo de clasificación de aprendizaje automático

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

## Modelos de aprendizaje automatico:

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [ ]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] =  model

In [ ]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [ ]:
fit_models['rc'].predict(X_test) 

array(['Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Buuu que mal', 'Buuu que mal', 'Buuu que mal', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal',
       'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bi

Evaluar y serializar el modelo

Otras metricas: precision, recall, f1 score, AUC. Aqui se usa precision

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) ## yhat almacena las predicciones reales  
    print(algo, accuracy_score(y_test, yhat)) #imprimimos la precision, 

lr 0.9939393939393939
rc 0.9939393939393939
rf 0.9878787878787879
gb 0.9818181818181818


Lo que se hace a continuacion tiene muchos casos de uso diferentes, se puede hacer una deteccion de accion o un modelo de lenguaje de señas(que se base en un solo cuadro o en una sola coordenada), analisis de sentimientos, etc.

In [ ]:
fit_models['rc'].predict(X_test)

array(['Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Buuu que mal', 'Buuu que mal', 'Buuu que mal', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal',
       'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bi

In [ ]:
y_test

435    Buuu que mal
20           Bieeen
11           Bieeen
129          Bieeen
348    Buuu que mal
           ...     
43           Bieeen
439    Buuu que mal
115          Bieeen
496    Buuu que mal
122          Bieeen
Name: class, Length: 165, dtype: object

In [ ]:
# Guardar el modelo ya entrenado
with open('modelo.pkl', 'wb') as f: 
    pickle.dump(fit_models['rf'], f) 

## Cargar el modelo ya entrenado

In [ ]:
with open('modelo.pkl', 'rb') as f: 
    model = pickle.load(f)

In [ ]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
         #                        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
          #                       mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
           #                      )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, ''
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, ''
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


In [78]:
results

mediapipe.python.solution_base.SolutionOutputs

In [79]:
results.pose_landmarks

landmark {
  x: 0.5809244
  y: 0.24712658
  z: -0.739278
  visibility: 0.99993914
}
landmark {
  x: 0.6049273
  y: 0.15868685
  z: -0.6839315
  visibility: 0.99988085
}
landmark {
  x: 0.6220694
  y: 0.15958098
  z: -0.68387604
  visibility: 0.99987334
}
landmark {
  x: 0.63892114
  y: 0.16115968
  z: -0.6841286
  visibility: 0.99987036
}
landmark {
  x: 0.54849285
  y: 0.15705356
  z: -0.6899874
  visibility: 0.9998952
}
landmark {
  x: 0.52555025
  y: 0.15668614
  z: -0.68933886
  visibility: 0.9998963
}
landmark {
  x: 0.50590986
  y: 0.15615633
  z: -0.689562
  visibility: 0.9999029
}
landmark {
  x: 0.6626643
  y: 0.2011099
  z: -0.35323778
  visibility: 0.9998439
}
landmark {
  x: 0.47565588
  y: 0.19750349
  z: -0.3682824
  visibility: 0.99991006
}
landmark {
  x: 0.6106254
  y: 0.33324215
  z: -0.61935574
  visibility: 0.9999331
}
landmark {
  x: 0.54190326
  y: 0.33145252
  z: -0.6230754
  visibility: 0.9999421
}
landmark {
  x: 0.77338624
  y: 0.67101187
  z: -0.19209817
  vi

In [80]:
results.pose_landmarks.landmark[0]

x: 0.5809244
y: 0.24712658
z: -0.739278
visibility: 0.99993914

In [81]:
results.pose_landmarks.landmark[0].visibility

0.9999391436576843

In [82]:
results.face_landmarks.landmark[0]

x: 0.56733704
y: 0.33550656
z: -0.0311259

In [83]:
results.face_landmarks.landmark[0].visibility # visibility muestra si el landmark si se muestra en la pantalla o no
# para los landmarks de la cara siempre sera cero

0.0

<img src="https://i.imgur.com/AzKNp7A.png">

## 2. Capturar Landmarks & Exportar CSV

In [84]:
import csv
import os
import numpy as np

In [103]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_coords #501

501

In [104]:
#Create header table | class | x | y | z | visibility |
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [105]:
len(landmarks) #2005

2005

In [106]:
#Export header CSV
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## 3. Tomar coordenadas de señas

In [108]:
#Save coord poses
class_name="Buuu que mal"

#Detectar puntos de referencia
cap = cv2.VideoCapture(0)  #0 o 2

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        # 1. Draw face landmarks
       # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
       #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
       #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
       #                          )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates, da todos los landmarks y los almacena en una matriz
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Concate rows
            row = pose_row+face_row

            # Append class name
            row.insert(0, class_name)

            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass


        cv2.imshow('Rosa Webcam', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [109]:
results.pose_landmarks.landmark[0]

x: 0.5739429
y: 0.31865352
z: -0.85075855
visibility: 0.9998646

In [121]:
results.face_landmarks.landmark[0].visibility #la visibilidad de face_landmark siempre sera cero

0.0

Numero de coordenadas que tendremos:

In [144]:
pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
len(pose_row) #132


132

In [157]:
face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
len(face_row)

1872

In [146]:
row=pose_row+face_row #total de coordenadas deberia de ser 2004
len(row)


2004

In [148]:
1+501*4

2005

In [147]:
row.insert(0, class_name) #agregar el nombre de la clase (pose)
len(row) #1+501*4 class_name, landmarks, header (x,y,z,visibility)

2005

## 4. Entrenar modelo usando Scikit Learn

### Leer y recolectar datos

In [158]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [159]:
df = pd.read_csv('coords.csv')

In [161]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Buuieeen,0.530443,0.256885,-0.653312,0.999986,0.559472,0.183092,-0.591619,0.999977,0.577272,...,0.003441,0.0,0.577638,0.191048,0.022638,0.0,0.582679,0.182326,0.023350,0.0
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0


In [162]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
542,Buuu que mal,0.561161,0.312676,-0.954139,0.999895,0.585262,0.240363,-0.879301,0.999777,0.600595,...,0.002703,0.0,0.612643,0.256506,0.021253,0.0,0.617767,0.247276,0.021986,0.0
543,Buuu que mal,0.561202,0.313547,-1.004511,0.999883,0.585302,0.240690,-0.925527,0.999742,0.600615,...,0.002702,0.0,0.616873,0.256113,0.021999,0.0,0.621815,0.247989,0.022703,0.0
544,Buuu que mal,0.564997,0.315829,-0.923724,0.999874,0.587699,0.241852,-0.852775,0.999715,0.603202,...,0.003340,0.0,0.620147,0.253019,0.022850,0.0,0.625026,0.244550,0.023674,0.0
545,Buuu que mal,0.570446,0.317763,-0.891257,0.999865,0.592185,0.243242,-0.811182,0.999696,0.608082,...,0.002278,0.0,0.621281,0.251643,0.021229,0.0,0.626135,0.244106,0.021871,0.0
546,Buuu que mal,0.573943,0.318654,-0.850759,0.999865,0.594787,0.243582,-0.771189,0.999693,0.610638,...,0.002614,0.0,0.622002,0.249436,0.022121,0.0,0.626846,0.241835,0.022813,0.0


In [165]:
df[df['class']=='Bieeen']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0
5,Bieeen,0.524074,0.269810,-0.796441,0.999970,0.551455,0.191224,-0.731950,0.999950,0.567993,...,0.003378,0.0,0.574250,0.191542,0.022691,0.0,0.579120,0.183588,0.023304,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,Bieeen,0.502346,0.364458,-0.711878,0.999970,0.523844,0.285156,-0.645986,0.999950,0.538261,...,0.000084,0.0,0.568144,0.287267,0.020360,0.0,0.573151,0.280079,0.021128,0.0
333,Bieeen,0.504117,0.367366,-0.690439,0.999969,0.525992,0.288800,-0.626301,0.999947,0.540995,...,0.000711,0.0,0.572651,0.290650,0.022053,0.0,0.578006,0.282941,0.022980,0.0
334,Bieeen,0.504405,0.366832,-0.775375,0.999964,0.526557,0.287651,-0.712337,0.999936,0.541663,...,0.001722,0.0,0.570707,0.279155,0.022135,0.0,0.576245,0.269708,0.023198,0.0
335,Bieeen,0.504425,0.360905,-0.769675,0.999963,0.527333,0.283022,-0.703782,0.999935,0.542733,...,0.000002,0.0,0.569716,0.265240,0.020307,0.0,0.575013,0.256650,0.021239,0.0


x, y, z son las caracteristicas que le pasaremos al modelo para entrenarlo.
La idea es que se pasen esas coordenadas y el modelo pueda predecir que clase es (pose)

In [167]:
X = df.drop('class', axis=1) # coordenadas
y = df['class'] # clase

In [172]:
y #X = coords , y = class

0          Buuieeen
1            Bieeen
2            Bieeen
3            Bieeen
4            Bieeen
           ...     
542    Buuu que mal
543    Buuu que mal
544    Buuu que mal
545    Buuu que mal
546    Buuu que mal
Name: class, Length: 547, dtype: object

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [174]:
X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
444,0.607627,0.331788,-0.786915,0.999879,0.629459,0.266417,-0.717988,0.999843,0.644549,0.268743,...,0.001988,0.0,0.665968,0.260837,0.021296,0.0,0.670673,0.253972,0.021934,0.0
480,0.561522,0.337758,-0.722480,0.999941,0.583754,0.270509,-0.622497,0.999925,0.598792,0.271359,...,0.004055,0.0,0.608456,0.254144,0.021859,0.0,0.613111,0.245462,0.022605,0.0
134,0.505146,0.288510,-0.693865,0.999984,0.529100,0.220908,-0.624870,0.999972,0.543936,0.223276,...,0.002321,0.0,0.554203,0.209690,0.022227,0.0,0.558938,0.202911,0.022755,0.0
93,0.512465,0.361475,-0.661181,0.999947,0.533841,0.292627,-0.601996,0.999917,0.547602,0.292967,...,0.001359,0.0,0.565009,0.273539,0.020338,0.0,0.569777,0.266539,0.020884,0.0
363,0.538592,0.274794,-0.750494,0.999549,0.565521,0.204791,-0.675919,0.999103,0.580280,0.206831,...,0.004653,0.0,0.594222,0.208484,0.024701,0.0,0.599227,0.200359,0.025415,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,0.488968,0.363467,-0.626467,0.999947,0.515220,0.283686,-0.562118,0.999918,0.531321,0.281666,...,-0.002455,0.0,0.550067,0.273457,0.016614,0.0,0.555085,0.266556,0.017167,0.0
372,0.553317,0.296857,-0.723984,0.999655,0.575554,0.220616,-0.667189,0.999247,0.591970,0.219673,...,0.003934,0.0,0.607926,0.218048,0.023417,0.0,0.612781,0.210321,0.024091,0.0
204,0.480797,0.354897,-0.773540,0.999975,0.500139,0.278386,-0.703254,0.999961,0.516196,0.275988,...,0.001624,0.0,0.530649,0.266843,0.021341,0.0,0.535038,0.260296,0.022014,0.0
53,0.520480,0.353810,-0.560121,0.999949,0.541462,0.283457,-0.492140,0.999928,0.554404,0.284279,...,0.002891,0.0,0.569414,0.277148,0.022025,0.0,0.574226,0.270020,0.022672,0.0


In [175]:
df #marco de datos completo, deben de ser 548 filas

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Buuieeen,0.530443,0.256885,-0.653312,0.999986,0.559472,0.183092,-0.591619,0.999977,0.577272,...,0.003441,0.0,0.577638,0.191048,0.022638,0.0,0.582679,0.182326,0.023350,0.0
1,Bieeen,0.527919,0.270350,-0.671225,0.999984,0.555597,0.191837,-0.610291,0.999974,0.572588,...,0.000889,0.0,0.575438,0.192714,0.018138,0.0,0.580191,0.184997,0.018685,0.0
2,Bieeen,0.526927,0.269982,-0.686023,0.999982,0.554501,0.191207,-0.624949,0.999969,0.571297,...,0.002067,0.0,0.574890,0.188843,0.020768,0.0,0.579669,0.182104,0.021323,0.0
3,Bieeen,0.526678,0.269773,-0.686642,0.999981,0.554088,0.190821,-0.628925,0.999967,0.570879,...,0.002285,0.0,0.575821,0.186804,0.020908,0.0,0.580568,0.179891,0.021366,0.0
4,Bieeen,0.525802,0.269881,-0.776611,0.999976,0.552998,0.191072,-0.715791,0.999958,0.569570,...,0.001017,0.0,0.574593,0.188575,0.019574,0.0,0.579291,0.181415,0.020120,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,Buuu que mal,0.561161,0.312676,-0.954139,0.999895,0.585262,0.240363,-0.879301,0.999777,0.600595,...,0.002703,0.0,0.612643,0.256506,0.021253,0.0,0.617767,0.247276,0.021986,0.0
543,Buuu que mal,0.561202,0.313547,-1.004511,0.999883,0.585302,0.240690,-0.925527,0.999742,0.600615,...,0.002702,0.0,0.616873,0.256113,0.021999,0.0,0.621815,0.247989,0.022703,0.0
544,Buuu que mal,0.564997,0.315829,-0.923724,0.999874,0.587699,0.241852,-0.852775,0.999715,0.603202,...,0.003340,0.0,0.620147,0.253019,0.022850,0.0,0.625026,0.244550,0.023674,0.0
545,Buuu que mal,0.570446,0.317763,-0.891257,0.999865,0.592185,0.243242,-0.811182,0.999696,0.608082,...,0.002278,0.0,0.621281,0.251643,0.021229,0.0,0.626135,0.244106,0.021871,0.0


### Modelo de clasificación de aprendizaje automático

In [176]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

## Modelos de aprendizaje automatico:

In [177]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [178]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [179]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [180]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] =  model

In [181]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [185]:
fit_models['rc'].predict(X_test) 

array(['Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Buuu que mal', 'Buuu que mal', 'Buuu que mal', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal',
       'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bi

Evaluar y serializar el modelo

Otras metricas: precision, recall, f1 score, AUC. Aqui se usa precision

In [186]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [187]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) ## yhat almacena las predicciones reales  
    print(algo, accuracy_score(y_test, yhat)) #imprimimos la precision, 

lr 0.9939393939393939
rc 0.9939393939393939
rf 0.9878787878787879
gb 0.9818181818181818


Lo que se hace a continuacion tiene muchos casos de uso diferentes, se puede hacer una deteccion de accion o un modelo de lenguaje de señas(que se base en un solo cuadro o en una sola coordenada), analisis de sentimientos, etc.

In [193]:
fit_models['rc'].predict(X_test)

array(['Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen',
       'Buuu que mal', 'Buuu que mal', 'Buuu que mal', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal',
       'Bieeen', 'Bieeen', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Buuu que mal', 'Bieeen', 'Bieeen', 'Buuu que mal', 'Buuu que mal',
       'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Buuu que mal', 'Bieeen',
       'Bieeen', 'Buuu que mal', 'Buuu que mal', 'Bieeen', 'Bieeen',
       'Bi

In [194]:
y_test

435    Buuu que mal
20           Bieeen
11           Bieeen
129          Bieeen
348    Buuu que mal
           ...     
43           Bieeen
439    Buuu que mal
115          Bieeen
496    Buuu que mal
122          Bieeen
Name: class, Length: 165, dtype: object

In [196]:
# Guardar el modelo ya entrenado
with open('modelo.pkl', 'wb') as f: 
    pickle.dump(fit_models['rf'], f) 

## Cargar el modelo ya entrenado

In [197]:
with open('modelo.pkl', 'rb') as f: 
    model = pickle.load(f)

In [198]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [199]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
         #                        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
          #                       mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
           #                      )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, ''
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, ''
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Buuu que mal [0. 1.]


: 